OS: Linux 6.1.85+

참고: Google Colab에서 실행했으며, 해당 환경은 리눅스 기반입니다.

Python: 3.10.12

pandas: 2.2.2

numpy: 1.26.4

matplotlib: 3.8.0

seaborn: 0.13.2

statsmodels: 0.14.4

pmdarima: 2.0.4

scikit-learn: 1.5.2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
import platform
import pmdarima
import sklearn
import os
import seaborn as sns
from pandas.tseries.offsets import Week
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error

In [ ]:
#print(f"OS: {platform.system()} {platform.release()}")
#print(f"참고: Google Colab에서 실행했으며, 해당 환경은 리눅스 기반입니다.")
#print(f"Python: {platform.python_version()}")
#print(f"pandas: {pd.__version__}")
#print(f"numpy: {np.__version__}")
#print(f"matplotlib: {matplotlib.__version__}")
#print(f"seaborn: {sns.__version__}")
#print(f"statsmodels: {sm.__version__}")
#print(f"pmdarima: {pmdarima.__version__}")
#print(f"scikit-learn: {sklearn.__version__}")

전처리

In [ ]:
# 2023년 데이터 불러오기
file_paths = ['/content/2023_raw.csv'] # 제공되는 데이터에 대해 코드 상에 데이터 입/출력 경로 포함 필요

# 데이터프레임에 저장
dfs = [pd.read_csv(file, encoding='utf-8') for file in file_paths]

# 데이터프레임 합치기 (현재는 단일 파일이라 사용 X)
merged_df = pd.concat(dfs, ignore_index=True)

# 승ㆍ하차 구분 열에서 하차인 데이터 삭제
merged_df = merged_df[merged_df['승ㆍ하차 구분'] != '하차']

# 승ㆍ하차 구분 열 삭제
merged_df.drop(columns=['승ㆍ하차 구분'], inplace=True)

# TX_STATUS 열 삭제
merged_df.drop(columns=['TX_STATUS'], inplace=True)

# 승차권 종류 열 삭제
merged_df.drop(columns=['승차권 종류'], inplace=True)

# BUSINESS_DAY 열을 날짜 형식으로 변환
merged_df['BUSINESS_DAY'] = pd.to_datetime(merged_df['BUSINESS_DAY'], format='%Y%m%d')

# BUSINESS_DAY와 STATION_NO 기준으로 그룹화하여 합산
agg_columns = [col for col in merged_df.columns if col not in ['BUSINESS_DAY', 'STATION_NO']]
merged_df = merged_df.groupby(['BUSINESS_DAY', 'STATION_NO'])[agg_columns].sum().reset_index()

# BUSINESS_DAY가 일요일인 데이터만 필터링
merged_df = merged_df[merged_df['BUSINESS_DAY'].dt.weekday == 6]  # 6은 일요일

# BUSINESS_DAY가 2023년 11월 26일까지만 필터링
merged_df = merged_df[merged_df['BUSINESS_DAY'] <= '2023-11-26']

# 인덱스 재설정
merged_df.reset_index(drop=True, inplace=True)

# 공백 열 삭제
merged_df = merged_df.drop(columns=['   '], errors='ignore')

# === 이상치 처리 ===
def handle_outliers_by_station(data, time_slots, end_date):
    stations = data['STATION_NO'].unique()

    filtered_data = data[data['BUSINESS_DAY'] <= end_date]

    # 각 역에 대해 이상치 탐지 및 처리
    for station in stations:
        station_data = filtered_data[filtered_data['STATION_NO'] == station]
        for time_slot in time_slots:
            # IQR 계산
            Q1 = station_data[time_slot].quantile(0.25) # 하이퍼파라미터: 0.25 (1분위수)
            Q3 = station_data[time_slot].quantile(0.75) # 하이퍼파라미터: 0.75 (3분위수)
            IQR = Q3 - Q1

            # 이상치 기준값 설정
            lower_bound = Q1 - 1.5 * IQR # 하이퍼파라미터: 1.5 (IQR 범위 계수)
            upper_bound = Q3 + 1.5 * IQR

            # 이상치 대체 (평균값으로)
            data.loc[(data['STATION_NO'] == station) & (data['BUSINESS_DAY'] <= end_date), time_slot] = np.where(
                (station_data[time_slot] < lower_bound) | (station_data[time_slot] > upper_bound),
                station_data[time_slot].mean(),
                station_data[time_slot]
            )
    return data

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 이상치 처리 실행
merged_df = handle_outliers_by_station(merged_df, time_slots, end_date='2023-11-26')

데이터프레임 확인

In [ ]:
merged_df

merged_df 기준 : 전체 역의 일별 승차량

In [ ]:
# BUSINESS_DAY 기준으로 TOTAL_CNT 합산
daily_totals = merged_df.groupby('BUSINESS_DAY')['TOTAL_CNT'].sum()

# 그래프 출력
plt.figure(figsize=(14, 7))
daily_totals.plot(kind='line', marker='o', color='g', linestyle='-', title='Daily Total Passenger Count')
plt.xlabel('Date')
plt.ylabel('Total Passenger Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

1101번~1122번 역 예측

In [ ]:
# 역 번호를 변수로 받기
station_no = 1101

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1102

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1103

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1104

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1105

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1106

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1107

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1108

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1109

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1110

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1111

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1112

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1113

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1114

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1115

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1116

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1117

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1118

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1119

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1120

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1121

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# 역 번호를 변수로 받기
station_no = 1122

# 시간대 리스트
time_slots = [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]

# 결과 저장용 데이터프레임 초기화
all_forecasts = pd.DataFrame()

# 각 시간대별로 예측 수행
for time_slot in time_slots:
    print(f"\n{station_no}역의 {time_slot} 예측 중...")

    # 데이터 준비
    station_data = merged_df[merged_df['STATION_NO'] == station_no]
    data = station_data.set_index('BUSINESS_DAY')[time_slot]
    data = data.sort_index()

    # 학습 데이터와 예측 데이터 분리
    train_data = data[:'2023-11-26']  # 학습 데이터: 2023년 11월 26일까지
    prediction_dates = pd.date_range(start='2023-12-03', end='2024-12-01', freq=Week(weekday=6))  # 예측 날짜: 2023년 12월 3일부터 2024년 12월 1일까지

    # 데이터 값이 모두 동일한지 확인 (동일하다면 해당 값으로 예측)
    if train_data.nunique() == 1:
        constant_value = train_data.iloc[0]
        print(f"{time_slot}의 모든 값이 동일함: {constant_value}")
        forecast_values_rounded = [constant_value] * len(prediction_dates)
    else:
        # auto_arima로 최적의 (p, d, q) 찾기
        print(f"\nauto_arima 실행 중... ({time_slot})")
        auto_model = auto_arima(
            train_data,
            seasonal=False, # 하이퍼파라미터: 계절성 여부 (False로 설정)
            stepwise=True, # 하이퍼파라미터: 단계별 탐색 여부 (True로 설정하여 효율적으로 탐색함)
            suppress_warnings=True, # 하이퍼파라미터: 경고 메시지 제거 여부
            error_action="ignore", # 하이퍼파라미터: 오류 발생 시 무시
            max_p=5, max_q=5, max_d=2  # 하이퍼파라미터: ARIMA 최대 (p, q, d) 값 설정
        )
        p, d, q = auto_model.order
        print(f"{time_slot}에서 최적의 (p, d, q): {p, d, q}")

        # ARIMA 모델 학습
        model = ARIMA(train_data, order=(p, d, q))
        fit_model = model.fit()

        # 미래 예측 (2023년 12월 3일부터 2024년 12월 1일까지)
        forecast_values = fit_model.forecast(steps=len(prediction_dates))

        # 소수점 첫째 자리에서 반올림하여 정수형으로 변환
        forecast_values_rounded = forecast_values.round(1).astype(int)

    # 예측 결과 데이터프레임 생성
    forecast_df = pd.DataFrame({
        'Date': prediction_dates,
        'Forecast': forecast_values_rounded,
        'Time_Slot': time_slot
    })

    # 결과 누적
    all_forecasts = pd.concat([all_forecasts, forecast_df], ignore_index=True)

# 2024년 12월 1일의 예측값만 저장
final_forecast = all_forecasts[all_forecasts['Date'] == '2024-12-01']
final_forecast.to_csv(f"forecast_dec_1_{station_no}.csv", index=False, encoding='utf-8')

# 시각화
for time_slot in time_slots:
    specific_forecast = all_forecasts[all_forecasts['Time_Slot'] == time_slot]
    plt.figure(figsize=(12, 6))
    plt.plot(
        merged_df[merged_df['STATION_NO'] == station_no].set_index('BUSINESS_DAY')[time_slot],
        label='Observed (Train)', color='blue'
    )
    # 모든 예측값을 빨간 점으로 표시
    plt.scatter(specific_forecast['Date'], specific_forecast['Forecast'], color='red', label='Forecast')
    plt.title(f'Forecast for {time_slot} for Station {station_no}')
    plt.xlabel('Date')
    plt.ylabel('Passenger Count')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

결과 Excel 파일 생성

In [ ]:
# 역 번호 리스트
station_numbers = range(1101, 1123)

# 통합 데이터 저장용 리스트
all_data = []

# 각 CSV 파일에서 데이터 읽기 및 통합
for station_no in station_numbers:
    csv_filename = f"/content/forecast_dec_1_{station_no}.csv" # 제공되는 데이터에 대해 코드 상에 데이터 입/출력 경로 포함 필요

    # CSV 파일 읽기
    if os.path.exists(csv_filename):
        df = pd.read_csv(csv_filename, encoding='utf-8')
        df['STATION_NO'] = station_no
        all_data.append(df)
    else:
        print(f"Warning: File {csv_filename} not found.")

# 모든 데이터를 하나로 통합
merged_df = pd.concat(all_data, ignore_index=True)

# 데이터 정리: BUSINESS_DAY, STATION_NO, CNT01~CNT00 순서로 열 정렬
merged_df = merged_df.pivot(index='STATION_NO', columns='Time_Slot', values='Forecast').reset_index()
merged_df.insert(0, 'BUSINESS_DAY', '20241201')

# 열 순서 정리: BUSINESS_DAY, STATION_NO, CNT01~CNT00
ordered_columns = ['BUSINESS_DAY', 'STATION_NO'] + [f"CNT{str(i).zfill(2)}" for i in range(1, 24)] + ["CNT00"]
merged_df = merged_df[ordered_columns]

# 열 이름 정리
merged_df.columns.name = None

# 최종 데이터 저장
output_filename = "prediction.csv"
merged_df.to_csv(output_filename, index=False, encoding='utf-8')

# 결과 확인
print("\nFinal Merged DataFrame:")
print(merged_df.head())

# 저장 파일 확인
print(f"\n저장 완료: {output_filename}")


In [ ]:
merged_df